In [1]:
import sys
from pathlib import Path
parent_dir = Path('.').resolve().parent
sys.path.insert(0, str(parent_dir))

import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

import methods


In [2]:
n = 10
columns = ['N', 'late_penality', 'waiting_charge', 'actual_time']
for i in range(1, n + 1):
    columns.append(f'a{i}')
df = pd.DataFrame(columns=columns)

In [3]:
for _ in range(10000):
    N = random.randint(n + 11, n + 39)
    intervals = np.random.gamma(shape=4, scale=2, size=N+n)

    # Variable gamma
    intervals = np.random.gamma(shape=random.randint(3, 7), scale=2, size=N+n)
    
    first_n = intervals[:n]
    late_penalty = random.randint(25, 500)
    waiting_charge = random.uniform(0.5, 1.8)
    actual_time = methods.cal_actual_time(n=5, intervals=intervals)
    dic = {'N': N, 'late_penality': late_penalty, 'waiting_charge': waiting_charge, 'actual_time': actual_time}
    for i in range(1, n + 1):
        dic[f'a{i}'] = first_n[i - 1]
    df.loc[len(df)] = dic
    # df.append(dic, ignore_index=True)


In [4]:
df.head()


,N,late_penality,waiting_charge,actual_time,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10
0,39,52,1.379168,587.545960,11.115910,8.246505,11.975258,7.690427,10.088621,17.622682,13.913990,24.278903,9.724588,11.143915
1,27,111,1.786848,464.013583,17.925308,14.608934,16.086399,14.787304,10.753037,17.678422,8.236334,12.154557,9.723966,9.296645
2,42,30,0.590746,531.898610,9.100083,9.964509,12.422146,17.172448,14.165024,11.625779,5.226312,14.897387,8.559861,17.333610
3,49,406,0.949083,295.567051,3.330323,3.452748,8.706291,8.211119,4.981542,9.666015,6.358240,6.395013,8.487680,3.945998
4,37,499,0.949871,582.856296,9.621048,6.385608,14.783582,21.649751,27.855731,15.369492,6.857356,15.801807,20.877417,13.105312


In [5]:
features = [x for x in columns if x != 'actual_time']
X = df[features]
y = df['actual_time']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
model = LinearRegression()

model.fit(X_train, y_train)
print(model.score(X_train, y_train))
model.coef_

0.8503564528139178


array([ 1.00412411e+01, -2.87588320e-03, -2.63607967e+00,  2.85054526e+00,
        2.71565539e+00,  2.81148013e+00,  2.70828243e+00,  2.82469623e+00,
        3.72434418e+00,  4.03297167e+00,  3.63886684e+00,  3.86848017e+00,
        3.74821219e+00])

In [7]:
print(model.score(X_test, y_test))

0.8518711470548117


In [8]:
gamma_shape = 4
gamma_scale = 2
N = 40
late_penalty = 100
waiting_charge = 1
costs = []
delays = []
for _ in range(10000):
    intervals = np.random.gamma(shape=4, scale=2, size=N+n)
    first_n = intervals[:n]
    dic = {'N': [N], 'late_penality': [late_penalty], 'waiting_charge': [waiting_charge]}
    for i in range(1, n + 1):
        dic[f'a{i}'] = [first_n[i - 1]]
    predicted_time = model.predict(pd.DataFrame(dic))[0] - 10
    actual_time = methods.cal_actual_time(n=n, intervals=intervals)
    cost = methods.cal_cost(c=late_penalty, h=waiting_charge, actual_time=actual_time, predicted_time=predicted_time)
    costs.append(cost)
    delays.append(actual_time - predicted_time)
methods.plot_plotly(data=costs, data_label='Cost')
methods.plot_plotly(data=delays, data_label='Delay')